License: LGPLv3

# Analysis of ChIP-Seq Data

In our last session, we introduced the concept of ChIP-Seq and discussed important experimental design aspects. Here, we will focus on the analytical steps after receiving your ChIP-Seq data from the sequencing facility. There are multiple options for almost every tool we discuss here; however, we chose to focus on the more popular software.

## Basic ChIP-Seq Workflow

<img src="assets/f04_chipseq_basic_workflow.png" alt="Figure 01" style="float: center;"/>

## Workflow Automation

There are numerous ways to automate an analytical pipeline. We are using the pipeline-building language called BigDataScript (https://pcingola.github.io/BigDataScript/). We do not have time to go over this language or our implementation; however, the source code is available for you to explore on your own time. We will go ahead and execute the pipeline to run in the background while we continue our discussion. Fortunately, jupyter notebooks allow us to easily run commands. **Only execute the cell below once!!** You will see an `*` in the column next to the cell while it's waiting for the task to complete.

In [15]:
%%bash
./pipelines/RunAmiaPipeline.sh

---------------------------------------------------------------------
2016 AMIA Pre-symposium Workshop
Center for Research Informatics, University of Chicago
Kyle Hernandez, Ph.D.

---------------------------------------------------------------------
---------------------------------------------------------------------
This script runs the ChIP-seq pipeline using BigDataScript and a small
subset of the data.

Writing main config: /home/ubuntu/dev/chipseq/CRI-Workshop-AMIA-2016-ChIPseq/workshop_extended/pipelines/config/chipseq.cfg
Writing Ab1-IgG contrast config: /home/ubuntu/dev/chipseq/CRI-Workshop-AMIA-2016-ChIPseq/workshop_extended/pipelines/config/contrast.Ab1.IgG.cfg
Writing Ab1-input contrast config: /home/ubuntu/dev/chipseq/CRI-Workshop-AMIA-2016-ChIPseq/workshop_extended/pipelines/config/contrast.Ab1.input.cfg
Writing Ab1 sample config: /home/ubuntu/dev/chipseq/CRI-Workshop-AMIA-2016-ChIPseq/workshop_extended/pipelines/config/sample.Ab1.cfg
Writing IgG sample config: /home/ubu

We will let this run (should only take a few minutes) and continue on with our overview using the pre-analyzed 
full dataset. Our data is taken from the published manuscript that focused on identifying novel tumor suppressors among the PR-domain (PRDM) family of genes. We will use two samples from this experiment, the Ab1 factor (for PRDM11) and RNA polymerase II. Unfortunately, they do not have any replicates; however, they do use both "input" 
and "IgG" controls. These are from U2932 B-cells for those who are interested.

## Raw Reads Processing

Here we assume that you already have your data de-multiplexed and processed from the sequencing center.

### Quality check of raw reads
First, we will run FastQC (http://www.bioinformatics.babraham.ac.uk/projects/fastqc/) on our raw fastq files. Since you have already been introduced to this tool and [MultiQC](http://multiqc.info/), we can go ahead and look at the results:

In [1]:
from IPython.display import IFrame
IFrame('pipelines/data/multiqc_report.html', width=1000, height=700)

### Read filtering and adapter removal

It is common practice to remove sequencing adapters and even filter/trim reads based on quality metrics. I would 
definitely recommend handling adapters and some light filtering of quality scores for ChIP-Seq samples. Here, we
will use [Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic) which can perform both utilities. Also, 
Trimmomatic comes with the commonly used adapter sequences which will work for most cases. 

If your raw data looks like it has some quality issues, I highly recommend re-running FastQC on your Trimmomatic-processed reads before continuing. 

## Genome Alignment

We will align our processed reads to the current build of the human genome (GRCh38). To do this we will use the 
[BWA aln](https://github.com/lh3/bwa) aligner. If you have reads that are longer than 70bp, then I would recommend 
that you use BWA mem instead. Regardless, before alignment you must create index files for your reference. You only
need to do this once and you can follow the instructions provided by the aligner. We indexed the reference before 
running the pipeline, so we can go ahead and run the aligner.

### Postprocessing alignments

Once you finish aligning, your BAM file (the type of binary file produced by aligners) contains unmapped reads and poorly aligned/ambiguous reads. Unless you _really_ want to keep all the information, it is common to do some filtering to remove unaligned reads and low-quality alignments. We will use the [sambamba](http://lomereiter.github.io/sambamba/) utility to do this.

After we filter our BAMs, we will sort (by genomic coordinate) and index them using sambamba. Indexing allows for rapid, random access that is key to downstream analyses. Next, we will use sambamba again to remove PCR duplicates. PCR duplicates can arise for multiple reasons and we *strongly* suggest that you remove them in ChIP-Seq libraries. They could lead to false positives or falsely inflated peaks. The signature of a PCR duplicate is when multiple reads have exactly the same start and stop location in the reference. These form stacks and the software attempts to locate these stacks and reduce them to only one read (see image below; [source](http://resources.qiagenbioinformatics.com/manuals/clcassemblycell/420/duplicate1.png).

<img src="assets/f05_chipseq_pcr_duplicate.png" alt="Figure 02" style="float: center;"/>

At this point, we will also use [deeptools](http://deeptools.readthedocs.io/en/latest/) to create bigWig files that
are all on the same scale for visualization later.

## Peak Calling

Once you have your filtered and PCR-dulicate removed alignments, we can move on to the peak detection step of our workflow. Here, we will use the popular tool [MACS2](http://liulab.dfci.harvard.edu/MACS/) to detect peaks. 

> Zhang et al. Model-based Analysis of ChIP-Seq (MACS). Genome Biol (2008) vol. 9 (9) pp. R137

Our experimental design uses 2 factors and uses 2 control samples (both input and IgG). We will pass our processed BAM files from our IP samples and each of the control samples to run MACS2 multiple times. So, we will end up running MACS2 a total of 4 times:

1. Ab1 w/ input
2. Ab1 w/ IgG
3. RNAPII w/ input
4. RNAPII w/ IgG

In addition, we will use another peak calling software called [Q](http://charite.github.io/Q/); however, we will only use the quality statistics it produces.

> Hansen et al. (2015) Saturation analysis of ChIP-seq data for reproducible identification of binding peaks. Genome Research, 25:1391-400.

After we detect our peaks, we will use [bedtools](http://bedtools.readthedocs.io/en/latest/) to get the peaks called by both the input and IgG controls:

1. Ab1 filtered peaks = (Ab1 vs. input) & (Ab1 vs. IgG)
2. RNAPII filtered peaks = (RNAPII vs. input) & (RNAPII vs. IgG)

### ChIP-Seq quality control

In addition to the basic read and alignment level quality control, there are some ChIP-Seq specific metrics to estimate the success/enrichment of your IP step. These metrics are discuss in detail in the [ENCODE guidelines](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3431496/) mentioned earlier. We will briefly go over 2 metrics: 1) relative strand correlation (RSC) and 2) the fraction of reads falling within peak regions (FRiP). We will get our RSC metrics from Q; however, we will estimate FRiP on our own.

#### RSC scores

<img src="assets/f06_chipseq_tag_shift.png" width="450px" alt="Figure 03" style="float: right;"/>

A high quality ChIP-Seq experiment produces significant clustering of enriched sequence tags at locations bound by the protein of interest. These sequence tags accumulate on both the forward and reverse strands centered around the binding site. 

These signal sequence tags are positioned at a distance from the binding site center that depends on the fragment size. The input control sequences lack this pattern of shifted stranded sequence tag densities. Cross-correlation metrics quantify the fragment clustering (IP enrichment) based on the correlation between genome-wide stranded tag densities. It is computed as the Pearson linear correlation between the strands after shifting the forward strand by _k_ base pairs. This produces two peaks when cross-correlation is plotted against the shift value:

1. A peak of enrichment corresponding to predominant fragment length
2. Peak corresponding to the read length (called a "phantom" peak) (Heinz et al. 2010)

> Heinz S, Benner C, Spann N, Bertolino E, Lin YC, Laslo P, Cheng JX, Murre C, Singh H, Glass CK
Mol Cell. 2010 May 28; 38(4):576-89.

<img src="assets/f07_chipseq_cc_plots.png" alt="Figure 04" style="float: center;"/>

The ratio between the fragment-length peak and the read-length peak is known as the relative strand correlation (RSC). $$RSC=\frac{cc\left(fragment\_length\right)-\min\left(cc\right)}{cc\left(read\_length\right)-\min\left(cc\right)}$$ RSC can be used for assessing the signal-to-noise ratio in ChIP-Seq experiments. High-quality ChIP-Seq data sets tend to have a larger fragment-length peak compared with the read-length peak, whereas failed ones and inputs have little or no such peak. According to ENCODE, **you should repeat samples with RSC values less than 0.8**.

##### Our data

The results from Q provide a cross-correlation plot with the RSC values; however, Q uses hamming distance in place of Pearson linear correlation. You can read their manuscript provided above for why they chose this metric. As you can see in the figures below, the same kind of patterns appear but are flipped upside down due to the use of hamming distance. Here, we see that both factors have a phantom peak just below 50bp on the x-axis (our read length is around 42bp), an estimated fragment length around 130, and RSC values well above the 0.8 cutoff, suggesting high enrichment.

<img src="assets/f08_chipseq_q_plots.png" alt="Figure 05" style="float: center;"/>

#### FRiP

Calculating the fraction of all mapped reads that fall into peak regions identified by a peak-calling algorithm (FRiP) is another metric to estimate global enrichment of ChIP-Seq data sets (Ji et al. 2008).

> Ji H, Jiang H, Ma W, Johnson DS, Myers RM, Wong WH
Nat Biotechnol. 2008 Nov; 26(11):1293-300.

Only a minority of reads in ChIP-Seq experiments occur in peaks with the rest representing background. ENCODE has shown that FRiP values correlate positively and linearly with the number of called regions as well as RSC values. In addition, **ENCODE suggests that you should repeat experiments with FRiP values below 1%**. They do note that these are just guidelines and it will ultimately depend on your factor and organism.

Unfortunately, Q does not provide FRiP, so we estimate it by dividing the number of reads in our detected MACS2 peaks by the number of alignmened reads in our processed BAM file. We use a combination of simple [bedtools](http://bedtools.readthedocs.io/en/latest/) and UNIX functions to perform the analysis.

##### Our data

Here, we only used the detected peaks that overlapped between the input and IgG controls. The results from our FRiP analysis are provided in the table below:

| Sample | Aligned Reads | Reads in Peaks | FRiP |
| -------| ------------- | -------------- | ---- |
| RNAPII | 27,742,292    | 3,704,044      | 0.13 |
| Ab1    | 19,895,204    | 467,872        | 0.02 |

As you can see, both our FRiP estimates are about the 1% suggested cutoff (especially the RNAPII sample!).

#### Other resources

1. ENCODE updated guidelines: https://www.encodeproject.org/about/experiment-guidelines/
2. ChIPQC - R/BioconductoR package for ChIP-Seq quality control; however, doesn't readily support GRCh38: https://bioconductor.org/packages/release/bioc/html/ChIPQC.html

## Visualizing peaks

Loading our normalized BigWig files and our narrowPeak files into [IGV](http://software.broadinstitute.org/software/igv/), we can get a better understanding of what we mean by "peaks" and the importance of control samples. Tracks:

1. narrowPeaks file from Ab1 (blue)
2. BigWig file from Ab1 (blue)
3. narrowPeaks file from RNAPII (red)
4. BigWig file from RNAPII (red)
5. BigWig file from IgG (grey)
6. BigWig file from input (grey)

We can clearly see the "peaks" in the IP samples relative to the two control samples.

<img src="assets/f10_peaks_IGV.png" alt="Figure 10" style="float: center;"/>

## BDS Summary

BDS provides a nice HTML report of the workflow. We can peek at the one we just produced from running the pipeline.

In [16]:
import os, glob
from IPython.display import IFrame

# Get the html file. It will be the only HTML file in the directory and we can glob for it
bds_report = glob.glob('pipelines/data/sample_run/AMIA-ChIP-Seq-Pipeline.bds.*.html')[0]

# Render HTML
IFrame(bds_report, width=1000, height=700)